# PINNs: a start

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import torch
import tensorflow as tf
import scipy.io
import torch.optim as optim
import time
from pinn import PINN
from net import Net
#from ns import PhysicsInformedNN

## Navier-Stokes equation

Given a two-dimensional velocity field, 

$$ {\bf V}(x, y; t)  =  ( u(x, y; t), v(x, y; t) )  \; \;,  $$
its components satisfy the following equations

$$ u_t + \lambda_1 ( u u_x + v u_y ) = -p_x + \lambda_2 ( u_{xx} + u_{yy}) $$
$$ v_t + \lambda_1 ( u v_x + v v_y ) = -p_y+ \lambda_2 ( v_{xx} + v_{yy}) $$
with $ p = p(x, y; t)$ being the pressure. The parameters $\lambda_1 $ and $\lambda_2$ are unknown.

We are interested in learning the parameters $\{ \lambda \} $ as well as the pressure $ p(x, y; t)$.

Solutions are searched in a set satisfying continuity equation $ \nabla \cdot {\bf V}(x, y; t) = 0 $, 

$$ u_x + v_y = 0  \; \;.$$

Defining a latent function $ \psi = \psi(x, y; t) $ such that (how crucial is the use of $\psi$?):
$$ u = \psi_y   \;, $$ 
$$ v = - \psi_x  \;, $$ 
the continuity equation is satistied. Given a set ${\cal S}$ of (noisy) measurements of the velocity field, 

$$    {\cal S} = \{ t^{(j)}, x^{(j)}, y^{(j)} , u^{(j)}, v^{(j)}   \}_{j=1}^{N}  \;, $$
we define

$$ f(x, y; t) \equiv u_t + \lambda_1 ( u u_x + v u_y )  + p_x - \lambda_2 ( u_{xx} + u_{yy}) \;,$$    
$$  g(x, y; t) \equiv v_t + \lambda_1 ( u v_x + v v_y ) + p_y - \lambda_2 ( v_{xx} + v_{yy})  \;,$$
and proceed by jointly approximating

$$ \left[   \psi(x, y; t) ;  p(x, y; t)  \right]  \;, $$
using a single neural network with two outputs.

The prior assumption is taking into account in another neural network (PINN) with two outputs:

$$ \left[   f(x, y; t) ;  g(x, y; t)  \right]  \;. $$

The parameters $\{ \lambda \} $ operate as well as the parameters of the neural networks:


$$ \left[   \psi(x, y; t) ;  p(x, y; t)  \right]  \;, $$
$$ \left[   f(x, y; t) ;  g(x, y; t)  \right]  \;, $$
that can be trained using a mean squared error loss:

$$  MSE \equiv \frac{1}{N} \sum_{j=1}^{N} \left[  \left( u( x^{(j)}, y^{(j)}, t^{(j)}) - u^{(j)} \right)^2 + \left( v( x^{(j)}, y^{(j)}, t^{(j)}) - v^{(j)} \right)^2  \right]  + \frac{1}{N} \sum_{j=1}^{N} \left[   |  u( x^{(j)}, y^{(j)}, t^{(j)}) |^2 +  |g( x^{(j)}, y^{(j)}, t^{(j)})|^2       \right]     $$





### Data

From https://github.com/maziarraissi/PINNs/blob/master/main/Data/cylinder_nektar_wake.mat

In [2]:
data = scipy.io.loadmat('data/cylinder_nektar_wake.mat')

In [3]:
print(data['t'].shape)
print(data['X_star'].shape)
print(data['U_star'].shape)
print(data['p_star'].shape)

(200, 1)
(5000, 2)
(5000, 2, 200)
(5000, 200)


In [4]:
U_star = data['U_star'] # N x 2 x T
p_star = data['p_star'] # N x T
t_star = data['t'] # T x 1
X_star = data['X_star'] # N x 2

In [5]:
N = X_star.shape[0]
T = t_star.shape[0]

In [6]:
# Rearrange Data 
XX = np.tile(X_star[:, 0:1], (1, T)) # N x T
YY = np.tile(X_star[:, 1:2], (1, T)) # N x T
TT = np.tile(t_star, (1, N)).T # N x T

In [7]:
print(XX.shape)
print(YY.shape)
print(TT.shape)

(5000, 200)
(5000, 200)
(5000, 200)


In [8]:
# Rearrange Data 
UU = U_star[:, 0, :] # N x T
VV = U_star[:, 1, :] # N x T
pp = p_star # N x T

In [9]:
print(UU.shape)
print(VV.shape)
print(pp.shape)

(5000, 200)
(5000, 200)
(5000, 200)


In [10]:
## Flattening
x = XX.flatten()[:, None] # NT x 1
y = YY.flatten()[:, None] # NT x 1
t = TT.flatten()[:, None] # NT x 1
    
u = UU.flatten()[:, None] # NT x 1
v = VV.flatten()[:, None] # NT x 1
p = pp.flatten()[:, None] # NT x 1

In [11]:
print(x.shape)
print(y.shape)
print(t.shape)
print(u.shape)
print(v.shape)
print(p.shape)

(1000000, 1)
(1000000, 1)
(1000000, 1)
(1000000, 1)
(1000000, 1)
(1000000, 1)


In [19]:
# Training Data    
N_train = 100

idx = np.random.choice(N*T, N_train, replace=False)
x_train = torch.Tensor(x[idx,:])
y_train = torch.Tensor(y[idx,:])
t_train = torch.Tensor(t[idx,:])
u_train = torch.Tensor(u[idx,:])
v_train = torch.Tensor(v[idx,:])

In [20]:
layers = [3, 20, 20, 20, 20, 20, 20, 20, 20]
#layers = [3, 20, 20]

model   = PINN(x_train, 
               y_train, 
               t_train, 
               u_train,
               v_train,
               layers_size= layers,    # indentify from where this 3 comes from  
               out_size = 2,     # psi and p
               params_list= None)


optimizer = optim.Adam(params= model.parameters(), 
                      lr= 0.1, 
                      weight_decay= 0.01)

t0 = time.time()

epochs = 10

for epoch in range(epochs):
    
    u_hat, v_hat, p_hat, f_u, f_v = model.net(x_train, y_train, t_train)
    loss_ = model.loss(u_train, v_train, u_hat, v_hat, f_u, f_v)
    loss_print  = loss_

    optimizer.zero_grad()   # Clear gradients for the next mini-batches
    loss_.backward()         # Backpropagation, compute gradients
    optimizer.step()

    t1 = time.time()


    ### Training status
    print('Epoch %d, Loss= %.10f, Time= %.4f' % (epoch, 
                                                loss_print,
                                                t1-t0))

Epoch 0, Loss= 0.9246909618, Time= 45.5933


KeyboardInterrupt: 